In [1]:
#Importing the libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from math import sqrt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
dataset = pd.read_excel('/Users/eupirate/Documents/DA_L7/L7_term3/L7_term3_Online Retail.xlsx',nrows=10000)
dataset.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   InvoiceNo    10000 non-null  object        
 1   StockCode    10000 non-null  object        
 2   Description  9958 non-null   object        
 3   Quantity     10000 non-null  int64         
 4   InvoiceDate  10000 non-null  datetime64[ns]
 5   UnitPrice    10000 non-null  float64       
 6   CustomerID   7709 non-null   float64       
 7   Country      10000 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 625.1+ KB


In [4]:
dataset.isnull().sum()

InvoiceNo         0
StockCode         0
Description      42
Quantity          0
InvoiceDate       0
UnitPrice         0
CustomerID     2291
Country           0
dtype: int64

In [5]:
dataset.dropna(subset=['CustomerID'],inplace=True)

In [6]:
dataset.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


In [7]:
dataset.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [8]:
#We can start with the recommendation system
#We convert the customerId into string to facilitate the algorithm later

dataset['CustomerID'] = dataset['CustomerID'].astype(str)

In [9]:
#We need to identify the values that are greater than 0 (if there is a return, it will show up as a negative number)
#We do this for price and units

dataset = dataset[(dataset['Quantity']>0) & (dataset['UnitPrice']>0)]

In [10]:
#Now we will create the User_item matrix --> Understanding the behaviour of the user regarding the items

user_item_matrix = dataset.pivot_table(index='CustomerID', columns = 'StockCode', values = 'Quantity', aggfunc = 'sum').fillna(0)
user_item_matrix.head()

StockCode,10002,10120,10125,10133,10135,11001,15034,15036,15039,16011,...,90200E,90210B,90214J,90214M,90214S,90214V,BANK CHARGES,C2,M,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12395.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
12427.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
12431.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12433.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12472.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0


In [11]:
#We create the similiarity matrix

user_similarity = cosine_similarity(user_item_matrix)

In [12]:
#Display the similarities in a dataframe to make it easier to read

user_similarity_df = pd.DataFrame(user_similarity, index = user_item_matrix.index, columns = user_item_matrix.index)

In [13]:
user_similarity_df.head()

CustomerID,12395.0,12427.0,12431.0,12433.0,12472.0,12557.0,12567.0,12583.0,12600.0,12647.0,...,18055.0,18074.0,18085.0,18109.0,18118.0,18144.0,18156.0,18168.0,18229.0,18239.0
CustomerID,,,,,,,,,,,,,,,,,,,,,
12395.0,1.000000,0.001016,0.0,0.120145,0.005424,0.0,0.007398,0.000251,0.045507,0.001354,...,0.000000,0.0,0.000000,0.000000,0.082169,0.0,0.000000,0.000000,0.000000,0.000000
12427.0,0.001016,1.000000,0.0,0.000000,0.097313,0.0,0.022310,0.003119,0.003743,0.016819,...,0.000000,0.0,0.000000,0.000000,0.064238,0.0,0.019784,0.000000,0.000000,0.000000
12431.0,0.000000,0.000000,1.0,0.000000,0.000000,0.0,0.008241,0.101972,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.001096,0.046814,0.010355
12433.0,0.120145,0.000000,0.0,1.000000,0.051052,0.0,0.273787,0.040969,0.000000,0.149572,...,0.004935,0.0,0.019917,0.010834,0.059327,0.0,0.123161,0.019822,0.000000,0.000000
12472.0,0.005424,0.097313,0.0,0.051052,1.000000,0.0,0.027789,0.124216,0.001537,0.214054,...,0.000000,0.0,0.008854,0.000000,0.059341,0.0,0.009476,0.000000,0.000000,0.124836


In [14]:
#Rating prediction using the matrix user_item

def predict_ratings_user_based(user_item_matrix, user_similarity):
    mean_user_rating = user_item_matrix.mean(axis = 1).values
    rating_diff = (user_item_matrix.values - mean_user_rating[:, np.newaxis])
    pred = mean_user_rating[:, np.newaxis] + user_similarity.dot(rating_diff)/np.array([np.abs(user_similarity).sum(axis=1)]).T
    return pred

In [15]:
#Predicted ratings

user_predicted_ratings = predict_ratings_user_based(user_item_matrix, user_similarity)
user_predicted_ratings

array([[-0.00583403, -0.00783887,  0.01907939, ..., -0.00533554,
         0.45671429,  0.43684397],
       [-0.00332743, -0.04794131, -0.02986444, ..., -0.04794131,
         0.07794544,  1.0896085 ],
       [ 1.35052984, -0.07462831, -0.07462831, ..., -0.0711302 ,
        -0.0682343 ,  0.05699184],
       ...,
       [-0.08964638, -0.12832839, -0.1205555 , ..., -0.12819959,
         0.27137129, -0.10542151],
       [ 2.12549299, -0.06664603, -0.06664603, ..., -0.06317765,
         0.02039569,  0.19378369],
       [-0.01317226, -0.0285752 , -0.03184675, ..., -0.03521847,
         0.22303415,  0.06616598]])

In [16]:
#Checking the performance of the algorithm

def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

print('User-user collaborative error:', rmse(user_predicted_ratings, user_item_matrix.values))

User-user collaborative error: 39.793287722573005
